In [42]:
%pip install redis



Note: you may need to restart the kernel to use updated packages.


In [43]:

import redis

r = redis.Redis(host='localhost', port=6379, db=0)

In [44]:
from counter import get_last_counter

bank_id = f"bank_{get_last_counter()}"
bank_id

'bank_51'

In [45]:
r.keys()

[b'memory_banks:v1::bank_51',
 b'faiss_index:v1::bank_50',
 b'faiss_index:v1::bank_40',
 b'memory_banks:v1::bank_45',
 b'faiss_index:v1::bank_47',
 b'faiss_index:v1::bank_42',
 b'memory_banks:v1::bank_49',
 b'faiss_index:v1::bank_46',
 b'memory_banks:v1::bank_44',
 b'faiss_index:v1::bank_38',
 b'faiss_index:v1::bank_41',
 b'memory_banks:v1::bank_38',
 b'memory_banks:v1::bank_39',
 b'memory_banks:v1::bank_48',
 b'faiss_index:v1::bank_39',
 b'memory_banks:v1::bank_50',
 b'memory_banks:v1::bank_41',
 b'memory_banks:v1::bank_46',
 b'memory_banks:v1::bank_47',
 b'memory_banks:v1::bank_42',
 b'faiss_index:v1::bank_44',
 b'faiss_index:v1::bank_48',
 b'memory_banks:v1::bank_43',
 b'faiss_index:v1::bank_49',
 b'memory_banks:v1::bank_40',
 b'faiss_index:v1::bank_45',
 b'faiss_index:v1::bank_51']

In [46]:
r.get(f"memory_banks:v1::{bank_id}")

b'{"identifier":"bank_51","provider_resource_id":"bank_51","provider_id":"faiss","type":"memory_bank","memory_bank_type":"vector","embedding_model":"all-MiniLM-L6-v2","chunk_size_in_tokens":512,"overlap_size_in_tokens":null}'

In [47]:
r.info()

{'redis_version': '7.4.1',
 'redis_git_sha1': 0,
 'redis_git_dirty': 0,
 'redis_build_id': '7fd3520e9c14a41b',
 'redis_mode': 'standalone',
 'os': 'Linux 6.8.0-49-generic x86_64',
 'arch_bits': 64,
 'monotonic_clock': 'POSIX clock_gettime',
 'multiplexing_api': 'epoll',
 'atomicvar_api': 'c11-builtin',
 'gcc_version': '11.4.0',
 'process_id': 4389,
 'process_supervised': 'systemd',
 'run_id': 'fd477aea774b4109d9b19beeb8eb6d46298a1c58',
 'tcp_port': 6379,
 'server_time_usec': 1733807551321122,
 'uptime_in_seconds': 155885,
 'uptime_in_days': 1,
 'hz': 10,
 'configured_hz': 10,
 'lru_clock': 5754303,
 'executable': '/usr/bin/redis-server',
 'config_file': '/etc/redis/redis.conf',
 'io_threads_active': 0,
 'listener0': {'name': 'tcp', 'bind': '-::1', 'port': 6379},
 'connected_clients': 3,
 'cluster_connections': 0,
 'maxclients': 10000,
 'client_recent_max_input_buffer': 8,
 'client_recent_max_output_buffer': 0,
 'blocked_clients': 0,
 'tracking_clients': 0,
 'pubsub_clients': 0,
 'watch

In [48]:
from json import loads
from IPython.display import display

index = loads(r.get(f"faiss_index:v1::{bank_id}"))
# Write index to JSON file
import json
with open('faiss_index.json', 'w') as f:
    json.dump(index, f, indent=2)
display(index.keys())
display(len(index['id_by_index']))

dict_keys(['id_by_index', 'chunk_by_index', 'faiss_index'])

8714

In [73]:
import os
from llama_stack_client import LlamaStackClient

# Test run
client = LlamaStackClient(
    base_url="http://localhost:5001",
)

response = client.memory.query(
    bank_id=bank_id,
    query=["What is William Falder's occupation?"],
    params={"max_chunks": 10},
)

display(response)

for chunk, score in zip(response.chunks, response.scores):
    print(chunk)
    print(score)


QueryDocumentsResponse(chunks=[Chunk(content=', sir--for good.\n\nCOKESON.  It\'s over a week since Davis went.  Now it won\'t do,\nFalder.  You\'re neglecting your work for private life.  I shan\'t\nmention about the party having called, but----\n\nFALDER.  [Passing into his room] Thank you, sir.\n\n     COKESON stares at the door through which FALDER has gone out;\n     then shakes his head, and is just settling down to write, when\n     WALTER How comes in through the outer Office.  He is a rather\n     refined-looking man of thirty-five, with a pleasant, almost\n     apologetic voice.\n\nWALTER.  Good-morning, Cokeson.\n\nCOKESON.  Morning, Mr. Walter.\n\nWALTER.  My father here?\n\nCOKESON.  [Always with a certain patronage as to a young man who\nmight be doing better]  Mr. James has been here since eleven o\'clock.\n\nWALTER.  I\'ve been in to see the pictures, at the Guildhall.\n\nCOKESON.  [Looking at him as though this were exactly what was to be\nexpected]  Have you now--ye--

Chunk(content=', sir--for good.\n\nCOKESON.  It\'s over a week since Davis went.  Now it won\'t do,\nFalder.  You\'re neglecting your work for private life.  I shan\'t\nmention about the party having called, but----\n\nFALDER.  [Passing into his room] Thank you, sir.\n\n     COKESON stares at the door through which FALDER has gone out;\n     then shakes his head, and is just settling down to write, when\n     WALTER How comes in through the outer Office.  He is a rather\n     refined-looking man of thirty-five, with a pleasant, almost\n     apologetic voice.\n\nWALTER.  Good-morning, Cokeson.\n\nCOKESON.  Morning, Mr. Walter.\n\nWALTER.  My father here?\n\nCOKESON.  [Always with a certain patronage as to a young man who\nmight be doing better]  Mr. James has been here since eleven o\'clock.\n\nWALTER.  I\'ve been in to see the pictures, at the Guildhall.\n\nCOKESON.  [Looking at him as though this were exactly what was to be\nexpected]  Have you now--ye--es.  This lease of Boulter\'s--

In [50]:
import pandas as pd

datasets = {
    "narrativeqa": "data/narrativeqa.parquet",
}

df = pd.read_parquet(datasets["narrativeqa"])
df['context_length'] = df['context'].apply(lambda x: len(x))
df.describe()

,context_length
count,47.000000
mean,282956.829787
std,198586.466477
min,45910.000000
25%,102942.500000
50%,244507.000000
75%,377520.500000
max,785669.000000


In [51]:
doc_ids = []
questions = []
for _, row in df.iterrows():
    for question in row["questions"]:
        questions.append(question)
        doc_ids.append(_)

print(len(questions))
print(len(doc_ids))
display(questions[:5])
doc_ids[:40]

1398
1398


['Who is the senior clerk at James How & Sons?',
 'Who was under suspicion for the forged check?',
 "What is William Falder's occupation?",
 'Who turns Falder over to the police?',
 "Who acts as Falder's lawyer?"]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [52]:
from tqdm import tqdm

top_k = 10

results = []

for i in tqdm(range(0, len(questions))):
    response = client.memory.query(
        bank_id=bank_id,
        query=[questions[i]],
        params={"max_chunks": top_k},
    )
    res = [int(chunk.document_id) for chunk in response.chunks]
    results.append(res)

results

100%|██████████| 1398/1398 [04:26<00:00,  5.24it/s]


[[0, 0, 17, 7, 55, 17, 0, 0, 7, 7],
 [0, 7, 0, 0, 17, 17, 1, 38, 38, 17],
 [0, 17, 44, 0, 22, 0, 7, 0, 38, 44],
 [38, 0, 0, 0, 0, 55, 38, 55, 55, 0],
 [0, 0, 12, 0, 0, 0, 0, 0, 0, 1],
 [0, 7, 55, 1, 7, 7, 7, 14, 52, 55],
 [0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
 [16, 16, 16, 0, 6, 31, 16, 16, 16, 45],
 [0, 0, 0, 36, 35, 8, 11, 41, 53, 50],
 [15, 15, 53, 36, 15, 15, 38, 15, 5, 5],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 55],
 [1, 1, 1, 1, 1, 0, 1, 1, 1, 55],
 [38, 12, 55, 1, 55, 55, 55, 0, 0, 1],
 [18, 18, 18, 18, 53, 1, 18, 50, 18, 53],
 [1, 0, 0, 1, 0, 1, 0, 47, 0, 1],
 [0, 0, 0, 0, 0, 11, 0, 0, 0, 7],
 [48, 43, 40, 0, 19, 38, 48, 38, 19, 40],
 [0, 17, 0, 0, 0, 0, 38, 0, 0, 55],
 [4, 0, 0, 0, 7, 7, 8, 6, 7, 7],
 [38, 0, 5, 27, 3, 55, 19, 9, 3, 7],
 [17, 17, 0, 0, 0, 17, 0, 17, 38, 17],
 [0, 0, 0, 0, 0, 17, 0, 0, 0, 17],
 [0, 38, 17, 0, 17, 7, 38, 7, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 55, 0, 17],
 [15, 44, 15, 27, 3, 9, 3, 45, 54, 15],
 [0, 0, 12, 0, 0, 0, 0, 0, 0, 17],
 [0, 0, 0, 0, 0, 0, 15, 54, 0, 0],
 [0,

In [68]:
import numpy as np

top_k = 3

targets = np.array(doc_ids)
display(targets.shape)
targets = np.expand_dims(targets, axis=1)
display(targets.shape)
targets = targets == res
display(targets.shape)
targets = targets.astype(float)
display(targets)

(1398,)

(1398, 1)

(1398, 5)

array([[1., 1., 0., 0., 0.],
       [1., 0., 1., 1., 0.],
       [1., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [69]:


res = np.array(results)
res = res[:, :top_k]
targets = targets[:, :top_k]
display(targets.shape)
display(res.shape)

(1398, 3)

(1398, 3)

In [70]:
temp = np.sum(targets, axis=1)
temp = np.divide(temp, top_k)
precision = np.sum(temp) / len(temp)
precision

np.float64(0.6402002861230329)

In [71]:
temp = np.logical_or.reduce(targets, axis=1)
temp = temp.astype(float)
recall = np.sum(temp) / len(temp)
recall

np.float64(0.740343347639485)

In [72]:
ranks = np.tile(np.arange(1, top_k + 1), (len(questions), 1)).astype(float)
temp = np.copy(targets)
for i in range(len(temp)):
    first_true = True
    for j in range(len(temp[i])):
        if temp[i,j]:
            if first_true:
                first_true = False
            else:
                temp[i,j] = False
reciprocal_ranks = temp / ranks
reciprocal_ranks = np.sum(reciprocal_ranks) / len(temp)
reciprocal_ranks = np.mean(reciprocal_ranks)
reciprocal_ranks

np.float64(0.6963519313304721)